In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

# 1. Custom 데이터셋 (입력/출력 쌍)
# 질문과 답변을 딕셔너리로 구성
data = {
    'input': ['안녕', '이름이 뭐야?', '뭐해?', '날씨 어때?', '배고파', '심심해', '너 몇 살이야?', '잘자', '고마워', '안녕히 계세요'],
    'output': ['안녕하세요', '나는 챗봇이야', '그냥 쉬고 있어', '맑고 좋아', '밥 먹어야지!', '놀아줄까?', '비밀이야', '좋은 꿈 꿔!', '천만에요', '또 만나요']
}
df = pd.DataFrame(data)

# 출력 문장에 시작과 끝 토큰 추가
# >> [start]부터 [end]까지가 답변이다"를 모델에게 알려주기 위해서
df['output'] = df['output'].apply(lambda x: '[start] ' + x + ' [end]')


# 2. Tokenizer 준비: 텍스트를 숫자로 바꾸는 Tokenizer
# > 컴퓨터는 글자를 이해 못하니까 숫자로 바꾸는 작업
# > tokenizer는 각 단어에 고유 번호를 부여해주는 도구
tokenizer_in = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_out = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_in.fit_on_texts(df['input'])
tokenizer_out.fit_on_texts(df['output'])

input_seq = tokenizer_in.texts_to_sequences(df['input'])
output_seq = tokenizer_out.texts_to_sequences(df['output'])

# 3. Padding: 시퀀스를 길이에 맞게 패딩
# > 문장마다 길이가 다르기 때문에 같은 길이로 맞춰주는 작업
# > 빈 칸은 0으로 채워져요.
# → 마치 “시험지 한 줄에 다 안 들어가면 빈칸에 0점 처리하는 것”과 비슷
max_len_in = max(len(i) for i in input_seq)
max_len_out = max(len(i) for i in output_seq)
input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_len_in, padding='post')
output_seq = tf.keras.preprocessing.sequence.pad_sequences(output_seq, maxlen=max_len_out, padding='post')

# 디코더 예측 타깃 시퀀스 (디코더의 출력은 한 칸씩(오른쪽) shift 되어 있음)
# 예)
# > 입력: [start] 안녕하세요
# > 타깃: 안녕하세요 [end]
target_seq = np.concatenate([output_seq[:, 1:], np.zeros((len(output_seq), 1))], axis=-1)


# 4. Positional Encoding 정의
# transformer는 순서를 모르기 때문에, 위치 정보를 따로 더해줘야 해요
# 이건 각 단어가 문장 내에서 몇 번째에 위치하는지 알려주는 값
# 비유: 포지셔널 인코딩은 마치 '나 이 문장의 첫 번째 단어야', '난 세 번째 단어야' 하고 스스로 이름표 붙이는 작업
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, pos, i, d_model):
        angles = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return pos * angles

    def positional_encoding(self, position, d_model):
        # numpy로 생성
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model
        )

        # 짝수 인덱스에는 sin, 홀수 인덱스에는 cos 적용
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


# 5. Transformer 기반 간단 챗봇 모델 정의
vocab_in = len(tokenizer_in.word_index) + 1
vocab_out = len(tokenizer_out.word_index) + 1
d_model = 128
num_heads = 4
dff = 256

class ChatBot(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.enc_embedding = tf.keras.layers.Embedding(vocab_in, d_model)
        self.dec_embedding = tf.keras.layers.Embedding(vocab_out, d_model)
        self.pos_encoding = PositionalEncoding(100, d_model)

        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.final = tf.keras.layers.Dense(vocab_out)

    def call(self, inputs): 
        enc_inputs = inputs['enc_inputs']
        dec_inputs = inputs['dec_inputs']

        enc_embed = self.enc_embedding(enc_inputs)
        enc_embed = self.pos_encoding(enc_embed)

        dec_embed = self.dec_embedding(dec_inputs)
        dec_embed = self.pos_encoding(dec_embed)

        attn_output = self.attention(dec_embed, enc_embed, enc_embed)
        ffn_output = self.ffn(attn_output)
        return self.final(ffn_output)


# 6. 모델 컴파일 및 학습
model = ChatBot()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.fit({'enc_inputs': input_seq, 'dec_inputs': output_seq}, target_seq, epochs=300, verbose=0)


# 7. 챗봇 응답 함수
def chat(sentence):
    seq = tokenizer_in.texts_to_sequences([sentence])
    seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len_in, padding='post')

    output = [tokenizer_out.word_index['[start]']]
    for _ in range(max_len_out):
        output_pad = tf.keras.preprocessing.sequence.pad_sequences([output], maxlen=max_len_out, padding='post')
        # 딕셔너리 형태로 predict 입력
        prediction = model.predict({'enc_inputs': seq, 'dec_inputs': output_pad}, verbose=0)
        pred_id = tf.argmax(prediction[0, len(output)-1]).numpy()
        if pred_id == tokenizer_out.word_index.get('[end]', 0):
            break
        output.append(pred_id)

    response = tokenizer_out.sequences_to_texts([output])[0]
    response = response.replace('[start]', '').replace('[end]', '').strip()
    print(f"Me: {sentence}\n챗봇: {response}\n")


# 8. 테스트
chat("안녕")
chat("이름이 뭐야?")
chat("심심해")
chat("고마워")
